In [1]:
import numpy as np
import pandas as pd
import talib

from datetime import datetime

for m in [np, pd, talib]:
    print(m.__name__, m.__version__)

numpy 1.25.2
pandas 2.0.3
talib 0.4.22


In [2]:
b = []
b.append(1)
b.append(2)
b.append(3)
b

[1, 2, 3]

In [3]:
b[0]

1

In [4]:
class Line:
    def __init__(self, max_length: int = 0):
        self._items = []
        self._max_length = max_length

    def max_length(self, length: int):
        self._max_length = length

    def push(self, item: object):
        self._items.append(item)
        if self._max_length > 0 and len(self._items) > self._max_length:
            self._items.pop(0)

    def pop(self):
        self._items.pop(0)

    def items(self):
        return self._items

    @property
    def cur(self):
        return self._items[-1]
    
    @property
    def pre(self):
        return self._items[-2]
    
    def __getitem__(self, index: int):
        print(type(index))
        if index > 0:
            raise IndexError("line index must less than 1")
        try:
            return self._items[index - 1]
        except Exception as e:
            raise IndexError("line index out of range")

    def __len__(self):
        return len(self._items)


In [5]:
p = 3
l = Line(p + 1)
l.push(10)
l.push(11)
l.push(9)

print(l[-1])
print(l[0])

<class 'int'>
11
<class 'int'>
9


In [6]:
l.pop()
print(l[0])

<class 'int'>
9


In [7]:
print(len(l))
l.push(12)
l.push(13)
l.push(14)
print(len(l))
print(l.items())

2
4
[9, 12, 13, 14]


In [8]:
l.cur, l.pre

(14, 13)

In [9]:
ll = Line()
ll.push(12.12)
ll.push(13.13)
ll.push(14.14)
ll.cur

14.14

In [10]:
ll[0], ll[-1]

<class 'int'>
<class 'int'>


(14.14, 13.13)

In [11]:
s = pd.Series(name='test')

In [12]:
s

Series([], Name: test, dtype: object)

In [13]:
s['1'] = 10
s['2'] = 20
s['3'] = 30
s['4'] = 40
s['5'] = 50

In [14]:
s

1    10
2    20
3    30
4    40
5    50
Name: test, dtype: int64

In [15]:
s.iloc[0:2]

1    10
2    20
Name: test, dtype: int64

In [16]:
# [-1:0]
s.iloc[-2:None]

4    40
5    50
Name: test, dtype: int64

In [17]:
# [-3:-1]
s.iloc[-4:-1]

2    20
3    30
4    40
Name: test, dtype: int64

In [18]:
d = pd.DataFrame()

In [19]:
d

""


In [214]:
import talib
from typing import Union, List, Dict, Tuple

from common.bar_utils import get_datetime

class TimeSeries:
    def __init__(self, series: pd.Series = None, max_length: int = 0):
        self._series = series
        if series is None:
            self._series = pd.Series()
        self._max_length = max_length

    def max_length(self, length: int):
        self._max_length = length

    def push(self, index: Union[datetime, str], item: object, sort: bool = False):
        # insert
        if isinstance(index, datetime):
            self._series[index] = item
        elif isinstance(index, str):
            self._series[self._datetime(index)] = item
        else:
            raise IndexError('TimeSeries index error')
        # sort by index
        if sort:
            self._series.sort_index(inplace=True)
        # drop the first item
        if self._max_length > 0 and len(self._series) > self._max_length:
            self.pop()

    def pop(self):
        self._series.drop(self._series.index[0], inplace=True)

    def index(self) -> List[pd.DatetimeIndex]:
        return self._series.index

    def items(self) -> List:
        return self._series.to_list()

    def _datetime(self, datetime_str: str, fmt: str = "%Y%m%d %H:%M:%S") -> datetime:
        return datetime.strptime(datetime_str, fmt)

    def __getitem__(self, i: Union[int, slice]):
        # index
        if isinstance(i, int):
            # param validate
            if i > 0:
                raise IndexError("TimeSeries index must less than 1")
            # get value by index
            try:
                return self._series[i - 1]
            except Exception as e:
                raise IndexError("TimeSeries index out of range")
        # slice
        elif isinstance(i, slice):
            # param validate
            if (i.start is not None and i.start > 0) or (i.stop is not None and i.stop > 0):
                raise IndexError("TimeSeries slice index must less than 1")
            # start
            start = None
            if i.start is not None and i.start <= 0:
                start = i.start - 1
            # stop
            stop = None
            if i.stop is not None and i.stop < 0:
                stop = i.stop
            # get series by slice
            try:
                return TimeSeries(self._series[start:stop])
            except Exception as e:
                raise IndexError("TimeSeries index out of range")
        else:
            raise IndexError('TimeSeries index error')

    def __len__(self) -> int:
        return len(self._series)


class TimeLine(TimeSeries):

    def __init__(self, series: pd.Series = None, max_length: int = 0):
        super().__init__(series, max_length)

    def SMA(self, period):
        return TimeSeries(talib.SMA(self._series, timeperiod=period))

    def EMA(self, period):
        return TimeSeries(talib.EMA(self._series, timeperiod=period))

    def RSI(self, period):
        return TimeSeries(talib.RSI(self._series, timeperiod=period))



In [215]:
t = TimeSeries(max_length=10)

In [216]:
t.push('20230202 12:20:12', 12)
t.push('20230202 12:20:13', 13)
t.push('20230202 12:20:14', 14)

In [217]:
t.items()

[12, 13, 14]

In [218]:
t.pop()

In [219]:
t.items()

[13, 14]

In [220]:
t[0], t[-1]

(14, 13)

In [221]:
t.index()

DatetimeIndex(['2023-02-02 12:20:13', '2023-02-02 12:20:14'], dtype='datetime64[ns]', freq=None)

In [222]:
#t.SMA(1)

In [223]:
b = TimeLine(max_length=10)
b

In [224]:
b.push('20000101 10:10:10', 10)
b.push('20000101 10:10:09', 9)
b.items()

[10, 9]

In [225]:
b.SMA(2)

In [226]:
b.EMA(2).items()

[nan, 9.5]

In [227]:
b.push('20000101 10:10:11', 11)
b.push('20000101 10:10:13', 13)
b.push('20000101 10:10:12', 12)

In [228]:
b.items()

[10, 9, 11, 13, 12]

In [229]:
b.SMA(3).items()

[nan, nan, 10.0, 11.0, 12.0]

In [230]:
b.SMA(3)[0]

12.0

In [231]:
b.RSI(3)

In [232]:
b.RSI(3)[0]

61.53846153846154

In [233]:
print(type(b.RSI(3)))
b.items()

<class '__main__.TimeSeries'>


[10, 9, 11, 13, 12]

In [234]:
b[-1:0]

In [236]:
b[-1:].items()

[13, 12]

In [237]:
b[-2:-1]

In [238]:
type(b[0:])

__main__.TimeSeries

In [239]:
type(b[0])

numpy.int64

In [240]:
b, len(b)

(<__main__.TimeLine at 0x121c75fa0>, 5)

In [242]:
b.RSI(4).items()

[nan, nan, nan, nan, 66.66666666666666]

In [243]:
b.RSI(4).index()

DatetimeIndex(['2000-01-01 10:10:10', '2000-01-01 10:10:09',
               '2000-01-01 10:10:11', '2000-01-01 10:10:13',
               '2000-01-01 10:10:12'],
              dtype='datetime64[ns]', freq=None)

In [244]:
b.index()

DatetimeIndex(['2000-01-01 10:10:10', '2000-01-01 10:10:09',
               '2000-01-01 10:10:11', '2000-01-01 10:10:13',
               '2000-01-01 10:10:12'],
              dtype='datetime64[ns]', freq=None)

In [253]:
s1 = pd.Series(name='ttt')

In [254]:
s1['1'] = 1

In [255]:
s1.index

Index(['1'], dtype='object')

In [256]:
s1.values

array([1])

In [257]:
type(s1.values)

numpy.ndarray

In [258]:
s1

1    1
Name: ttt, dtype: int64